In [142]:
from bs4 import *
import re
import requests as rq

In [143]:
url = "https://pokemondb.net/pokedex/all"
page_contents = rq.get(url)
soup = BeautifulSoup(page_contents.text, 'html.parser')


In [144]:
url_sprites = 'https://pokemondb.net/pokedex/shiny'
page_contents_sprites = rq.get(url_sprites)
soup_sprite = BeautifulSoup(page_contents_sprites.text, 'html.parser')
print('new')

new


In [145]:
mon_sprites_data = []
mon_sprites = []
for link in soup_sprite.findAll('div'):
    if 'infocard' in link['class']:
        for i in link.children:
            try:
                print(i.small.text[1:])
                mon_sprites_data.append(i.text)
                print(i.text)
            except Exception as e:
                print(i)
                if i != '\n':
                    for j in i.contents:
                        if str(j) != '\n' and re.match(r'^<span class="img-fixed shinydex-sprite*', str(j)):
                            mon_sprites.append(str(j))



<span class="infocard-lg-img">
<span class="img-fixed shinydex-sprite shinydex-sprite-normal" data-alt="Bulbasaur normal sprite" data-src="https://img.pokemondb.net/sprites/home/normal/bulbasaur.png" data-srcset="https://img.pokemondb.net/sprites/home/normal-2x/jpg/bulbasaur.jpg 2x" style="width:128px;height:128px"></span> <span class="img-fixed shinydex-sprite shinydex-sprite-shiny" data-alt="Bulbasaur shiny sprite" data-src="https://img.pokemondb.net/sprites/home/shiny/bulbasaur.png" data-srcset="https://img.pokemondb.net/sprites/home/shiny-2x/jpg/bulbasaur.jpg 2x" style="width:128px;height:128px"></span> </span>


001

#001
Bulbasaur
 Grass · Poison





<span class="infocard-lg-img">
<span class="img-fixed shinydex-sprite shinydex-sprite-normal" data-alt="Ivysaur normal sprite" data-src="https://img.pokemondb.net/sprites/home/normal/ivysaur.png" data-srcset="https://img.pokemondb.net/sprites/home/normal-2x/jpg/ivysaur.jpg 2x" style="width:128px;height:128px"></span> <span class="

In [146]:
type_names = ["Normal", "Fire", "Water", "Electric", "Grass", "Ice", "Fighting", "Poison", "Ground", "Flying",
              "Psychic", "Bug", "Rock", "Ghost", "Dragon", "Dark", "Steel", "Fairy"]

In [147]:
def parse_dex_data(string):
    string = string.split('\n')[1:-1]
    typing = string[-1].split(' · ')
    for i in range(len(typing)):
        typing[i] = typing[i].strip(' ')
    dex_id = string[0][1:]
    name = string[1] + " " + string[2]
    for t in type_names:
        if t in string[2]:
            name = string[1]

    return name, dex_id, typing

def parse_image_data(string1, string2):
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    normal = re.search(url_regex,string1)
    shiny = re.search(url_regex, string2)

    return string1[normal.start():normal.end()], string2[shiny.start():shiny.end()]

In [148]:
for i in range(len(mon_sprites_data)):
    mon_sprites_data[i] = parse_dex_data(mon_sprites_data[i])
for i in range(0, len(mon_sprites)-1, 2):
    split_links = parse_image_data(mon_sprites[i], mon_sprites[i+1])
    mon_sprites[i]= split_links[0]
    mon_sprites[i+1] = split_links[1]



In [149]:
mon = []
soup.prettify()
for i in soup.find_all('tr'):
    mon.append(i.text)

In [150]:
def print_list(objects):
    for i in objects:
        print(i)


def get_types(line):
    types = []
    for t in type_names:
        if t in line:
            types.append(t)
    return types

def cut_types_from_string(line, types):
    for t in types:
        line = line.replace(t,'')
    return line


def get_dex_id(line):
    did_str = ''
    i = 0
    while line[i] != ' ':
        did_str += line[i].strip('\n')
        i += 1
    return did_str


def get_name(line):
    name = ""
    i = 4 # 4 is the start index of the
    while line[i] != '\n':
        name += line[i]
        i += 1
    return name[1:-1]

def parse_line(line):
    did = get_dex_id(line)
    types = get_types(line)
    line = cut_types_from_string(line, types)
    data = line.split('\n')
    name = get_name(line)
    base_stat_total = data[2]
    hp = data[3]
    attack = data[4]
    defense = data[5]
    sp_attack = data[6]
    sp_def = data[7]
    speed = data[8]
    print(did, name, types, base_stat_total, hp, attack, defense, sp_attack, sp_def, speed)
    return name, did, types, base_stat_total, hp, attack, defense, sp_attack, sp_def, speed


# returns string for Pokemon constructor with the data given
def objectify_data(data):
    if len(data) > 10:
        return 'Pokemon("' + str(data[0]) + '", "' + str(data[1]) + '", ' + str(data[2]) + ', "' + str(data[3]) + '", "' + str(data[4]) + '", "' + str(data[5]) + '", "' + str(data [6]) + '", "' + str(data[7]) +'", "' + str(data[8]) + '", "' + str(data[9]) + '", "' + str(data[10]) + '", "' + str(data[11]) + '")'
    return 'Pokemon("' + str(data[0]) + '", "' + str(data[1]) + '", ' + str(data[2]) + ', "' + str(data[3]) + '", "' + str(data[4]) + '", "' + str(data[5]) + '", "' + str(data [6]) + '", "' + str(data[7]) +'", "' + str(data[8]) + '", "' + str(data[9]) + '")'

In [151]:
mon_data = []
for i in range(1, len(mon)):
    mon_data.append(parse_line(mon[i]))


001 Bulbasaur ['Grass', 'Poison'] 318 45 49 49 65 65 45
002 Ivysaur ['Grass', 'Poison'] 405 60 62 63 80 80 60
003 Venusaur ['Grass', 'Poison'] 525 80 82 83 100 100 80
003 Venusaur Mega Venusaur ['Grass', 'Poison'] 625 80 100 123 122 120 80
004 Charmander ['Fire'] 309 39 52 43 60 50 65
005 Charmeleon ['Fire'] 405 58 64 58 80 65 80
006 Charizard ['Fire', 'Flying'] 534 78 84 78 109 85 100
006 Charizard Mega Charizard X ['Fire', 'Dragon'] 634 78 130 111 130 85 100
006 Charizard Mega Charizard Y ['Fire', 'Flying'] 634 78 104 78 159 115 100
007 Squirtle ['Water'] 314 44 48 65 50 64 43
008 Wartortle ['Water'] 405 59 63 80 65 80 58
009 Blastoise ['Water'] 530 79 83 100 85 105 78
009 Blastoise Mega Blastoise ['Water'] 630 79 103 120 135 115 78
010 Caterpie ['Bug'] 195 45 30 35 20 20 45
011 Metapod ['Bug'] 205 50 20 55 25 25 30
012 Butterfree ['Flying', 'Bug'] 395 60 45 50 90 80 70
013 Weedle ['Poison', 'Bug'] 195 40 35 30 20 20 50
014 Kakuna ['Poison', 'Bug'] 205 45 25 50 25 25 35
015 Beedrill 

In [152]:
FILE_PATH = 'pokemonObjectList.py'
print(len(mon_sprites))
print(len(mon_sprites_data))
print(len(mon_data))

2084
1042
1045


In [153]:
def compile_data(data, sprite_links, sprite_data):
    for i in range(0, len(sprite_links)-1, 2):
        sprite_data[i//2] = sprite_data[i//2] + (sprite_links[i], sprite_links[i+1],)
    for i in range(len(data)):
        for j in range(len(sprite_data)):
            if sprite_data[j][0] == data[i][0] and sprite_data[j][1] == data[i][1] and (sprite_data[j][2] == data[i][2] or data[i][2].reverse() == sprite_data[j][2]):
                data[i] = data[i] + (sprite_data[j][-2], sprite_data[j][-1],)
                break
    return data
mon_data = compile_data(mon_data, mon_sprites, mon_sprites_data)

In [154]:
def write_to_file(data_list):
    with open(FILE_PATH, 'w', encoding='UTF-8') as f:
        f.write("from Pokemon import *\n\n\nMON_LIST = [")
        for data in data_list:
            f.write(objectify_data(data) + ",\n")
        f.write("]")

In [155]:
print(objectify_data(mon_data[23]))
write_to_file(mon_data)

Pokemon("Pidgeot Mega Pidgeot", "018", ['Normal', 'Flying'], "579", "83", "80", "80", "135", "80", "121")
